# Empowerment Congress West Area

Simple idioms I use to display with ipyleaflet.

In this notebook we'll use one of the attributes of the 311 data (nc) to focus on the map.

Steps:

1.  Initial imports
2.  Read data
3.  Explore the data
4.  Visualize/Interact with the map
5.  Sense-making

# 1. Setup

I'm assuming you have an environment ala the environment.yml file.  

I have a startup script in the ~/.ipython/profile_default/startup directory.  I've copied it to this folder so you can run it for the same effect.

**Note:** I'm importing the utility function read_new311_shape to read the preproccessed 311 data and map the column names/types.

In [ ]:
%run start.py
import nc

from utils import read_new311_shape, marker_color_map

Next it's nice to get a sense about this Neighborhood Council.  I found this url linked off empowerla.org site.

They allow me to visualize as embedded IFrame so here it is.  This helps in storytelling.

In [ ]:
IFrame("https://ecwandc.org/about-ecwa", width=1400, height=800)

# 2. Get the Data

Standard idioms for this:

  > - 311 observations as a shape file from zip file
  > - NC polygons
  
Both of these data sets have been through the appropriate preprocessing steps.

In [ ]:
%%time
ecwandc311_gdf = read_new311_shape('../data/311/ecwandc311.shp')

**Note** - This is becoming a standard way to read the NC polygons.  Probably time to refactor and move to src?

In [ ]:
ncs_gdf = gpd.read_file('../data/neighborhoods/Neighborhood_Councils_(Certified)_cleaned.shp')

In [ ]:
ncs_gdf.rename(columns={'NAME': 'name',
                        'NC_ID': 'nc_id',
                        'SERVICE_RE': 'service_region'},
              inplace=True);

# 3. Examine the Data

In [ ]:
ecwandc311_gdf.columns

In [ ]:
ecwandc311_gdf.request_type.value_counts()

In [ ]:
ecwandc311_gdf.request_type.value_counts()

In [ ]:
ecwandc311_gdf['owner'].value_counts()

In [ ]:
ecwandc311_gdf.created_dt.min()

In [ ]:
ecwandc311_gdf.created_dt.max()

In [ ]:
(_ - __).days / 7

In [ ]:
import numpy as np
markers = list()

def closed(dt):
    if not(pd.isnull(dt)):
        return dt.strftime("%m/%d/%Y, %H:%M")
    else:
        return "Still Open"

for i, row in tqdm(ecwandc311_gdf.iterrows()):
    
    fill_color = marker_color_map[row.request_type]
    marker = CircleMarker(location=(row.geometry.y, row.geometry.x), radius=5, stroke=False, fill_color=fill_color, fill_opacity=1.0)
    msg = HTML()
    msg.value = "report: {}<br>Address: {}<br>when: {}<br>finished: {}<br>type: {}".format(row['SRNumber'], 
                                                              row['address'],
                                                            row['created_dt'].strftime("%m/%d/%Y, %H:%M"),
                                                            closed(row['closed_dt']),
                                                            row['request_type']) #"status: {}<br/>coord: {}".format(r['status'], r['coordinates'])
    marker.popup = msg
    markers.append(marker)
    ecwandc311_gdf.loc[i, 'marker'] = marker

ecwandc_311_cluster = MarkerCluster(markers=markers, name='311 Calls')


In [ ]:
#ncs_gdf.to_crs(epsg=4326, inplace=True)
ecwandc_gdf = ncs_gdf.query(f"nc_id == 79").reset_index()

In [ ]:
center = ecwandc_gdf.iloc[0].geometry.centroid.y, ecwandc_gdf.iloc[0].geometry.centroid.x

In [ ]:
nc_layer = GeoData(geo_dataframe = ecwandc_gdf,
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.8, 'weight':1.9, 'dashArray':'5', 'fillOpacity':0.2},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Empowerment Congress West Area')

In [ ]:
#from ipyleaflet import FullScreenControl
imagery = basemap_to_tiles(basemaps.Esri.WorldImagery)
imagery.base = True
osm = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
osm.base = True

google_map = TileLayer(
    url="https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}",
    attribution="Google",
    name="Google Maps",
)
google_map.base = True

google_satellite = TileLayer(
    url="https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}",
    attribution="Google",
    name="Google Satellite"
)
google_satellite.base = True

map_display = Map(center=center, zoom=15,
                  layers=[google_satellite, google_map, imagery, osm],
                  layout=Layout(height="900px"),
                  scroll_wheel_zoom=True)

map_display.add_control(LayersControl())

#map_display.add_layer(geo_data)
map_display.add_control(FullScreenControl())
map_display += nc_layer
map_display.add_layer(ecwandc_311_cluster)

all_311 = LayerGroup(name=f"All 311", layers=markers)
map_display += all_311
map_display

In [ ]:
neighborhood_census_df = pd.read_csv('../data/neighborhoods/Census_Data_by_Neighborhood_Council.csv')

In [ ]:
neighborhood_census_df.columns

In [ ]:
neighborhood_census_df['NC_Name'].to_list()

In [ ]:
neighborhood_census_df.query(f"NC_Name == 'EMPOWERMENT CONGRESS WEST AREA NDC'")

In [ ]:
neighborhood_census_df.iloc[21]

In [ ]:
(_['In_Poverty'] / _['Total Population'])

In [ ]:
__['White_pop'] / __['Total Population']

In [ ]:
___['Owner_occ'] / ___['Total Population']

In [ ]:
ecwandc311_gdf.columns

In [ ]:
ecwandc311_gdf.days_to_close.mean()